## Extract Data from Emails - Part 1 of 2
Extract information from Outlook emails using Win32 (and other applicable methods such as regex). The code for this project consists of two parts. The first part creates the initial file after going through all the emails in inbox.

### 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
import shutil
import win32com.client
import re
import time
import dateutil.parser
from dateutil import parser
from dateutil import relativedelta
from datetime import datetime
from pandas import ExcelWriter
from pandas import ExcelFile

### 2. Initiate

In [2]:
# Set Outlook settings to retrieve messages from inbox.
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
inbox = outlook.GetDefaultFolder(6) # For main inbox. To search within a subfolder add .Folders.Item("subfolder name").
messages = inbox.Items

# Set initial lists and dataframes.
submission = []
df_preval = pd.DataFrame()
df_prefain = pd.DataFrame()
df_error = pd.DataFrame()
df_errfain = pd.DataFrame()
df_cert = pd.DataFrame()
df_certfain = pd.DataFrame()

# Create a function to set agency based on file name.
def set_agency(row):
    if 'AM00' in str(row['file_name']):
        return 'Agricultural Marketing Service'
    elif 'AO00' in str(row['file_name']):
        return 'Office of Advocacy and Outreach'
    elif 'AP00' in str(row['file_name']):
        return 'Animal and Plant Health Inspection Service'
    elif 'AP02' in str(row['file_name']):
        return 'Federal Shared Service Provider'
    elif 'AR00' in str(row['file_name']) or 'AROO' in str(row['file_name']):
        return 'Agricultural Research Service'
    elif 'EC00' in str(row['file_name']):
        return 'Office of the Chief Economist'
    elif 'ER00' in str(row['file_name']):
        return 'Economic Research Service'
    elif 'FA00' in str(row['file_name']):
        return 'Farm Service Agency'
    elif 'FI00' in str(row['file_name']):
        return 'Food Safety and Inspection Service'
    elif 'FN00' in str(row['file_name']):
        return'Food and Nutrition Service'
    elif 'FS00' in str(row['file_name']):
        return 'Forest Service'
    elif 'FX00' in str(row['file_name']):
        return 'Foreign Agricultural Service'
    elif 'NA00' in str(row['file_name']):
        return 'National Agricultural Statistics Service'
    elif 'NI00' in str(row['file_name']):
        return 'National Institute of Food and Agriculture'
    elif 'NR00' in str(row['file_name']):
        return 'Natural Resources Conservation Service '
    elif 'RD00' in str(row['file_name']):
        return 'Rural Development'
    elif 'RM00' in str(row['file_name']):
        return 'Risk Management Agency'
    else:
        return ''

# Function to create agency code and agency columns. 
def agency_col(df):
    # Extract agency code from file name.
    df['agency_code'] = df['file_name'].str.extract('_(.*?)_', expand=True)
    # Apply set_agency function.
    df['agency'] = df.apply(set_agency, axis=1)

### 3. Loop Through and Set
Loop through and identify D2-related messages and set parameters and values for:

-  File submissions
-  Pre-validation errors
-  D2 error reports
-  D2 certifications

In [3]:
# Loop through each message and filter accordingly.
for m in messages:
    #Extract date.
    date_str = m.CreationTime.strftime('%Y-%m-%d %H:%M:%S')
    date = dateutil.parser.parse(date_str)
    
    # Retrieve file submission messages.
    if m.Subject.upper() == 'DATA ACT FILE SUBMISSION' and m.SenderName == 'OCFO - FMMI BI TEAM' or m.Subject.upper() == 'DATA ACT FILE SUBMISSION' and m.SenderName == 'FMMIBITEAM@cfo.usda.gov':
        # Set file name and date received and agency.
        date_submission = date
        submission_file_name = re.search(r'file (.*?) has', m.Body).group(1)
        # Append all records to submission list.
        submission.append({'file_submission_notification_date': date_submission,'file_name': submission_file_name})
        # Create dataframe.
        df_submission = pd.DataFrame(submission)
   
    # Retrieve pre-validation error messages.
    if m.Subject.upper() == 'DATA ACT D2 FILE SUBMISSION - PRE-VALIDATION ERROR' and  m.SenderName == 'BEREMOTE' or m.Subject.upper() == 'FW: DATA ACT D2 FILE SUBMISSION - PRE-VALIDATION ERROR' and m.SenderName == 'Smith, John - OCFO, Washington, DC':
        # Set date based on email receipt, i.e., directly or forwarded.
        if m.Subject.upper() == 'DATA ACT D2 FILE SUBMISSION - PRE-VALIDATION ERROR' and m.SenderName == 'BEREMOTE':
            date_preval = date
        else:
            # Extract the sent date from the body of the forwarded email.
            date_preval_text = re.search(r'day, (.*?)\n', m.Body).group(1)
            # Split at "(" for date formats containing (UTC)...
            date_preval_text = date_preval_text.split(' (')[0]
            # Convert date string to date-time.
            date_preval = parser.parse(date_preval_text)
        
        # Set filename and filepath and save to current directory.
        attachment = m.Attachments.Item(1)
        preval_attachment = attachment.FileName
        attachment.SaveAsFile(os.getcwd() + '\\' + preval_attachment)
        preval_file_name = preval_attachment[:-10]
        
        # Open text file with utf8 to prevent encoding errors. 
        with open (preval_attachment, 'rt', encoding='utf8') as txt_file:
            preval_content = txt_file.read()
            # Count instances of 'Error on line' in the text file.
            preval_count = len(re.findall(r'Error on line:', preval_content, re.IGNORECASE))
            # Split the text at "Error on line" for FAIN extraction.
            preval_text = preval_content.split("Error on line:",preval_count)[1:preval_count+1]
            
            # Loop through each resulting list element from the preceding split to extract the FAIN and action date.
            for string in preval_text:
                try:
                    # Extract data elements after splitting at commas.
                    preval_actdate = string.split(",")[5].strip() # Action date
                    preval_fain = string.split(",")[8].strip() # FAIN
                    preval_mod = string.split(",")[9].strip() # Award modification
                    preval_uri = string.split(",")[10].strip() # URI
                    preval_cfda = string.split(",")[32].strip() # CFDA
                except Exception:
                    preval_actdate = ''
                    preval_fain = ''
                    preval_mod = ''
                    preval_uri = ''
                    preval_cfda = ''
                
                #Convert to dataframe and set index=[0] to prevent scalar value error.
                prefain = pd.DataFrame({'file_name': preval_file_name, 'notification_date': date_preval, 'action_date': preval_actdate, 'fain': preval_fain, 'award_modification': preval_mod, 'uri': preval_uri, 'cfda_number': preval_cfda}, index=[0])
                # Append all dataframes created by the for loop.
                df_prefain = df_prefain.append(prefain)
                
        # Apply agency_col function on the dataframe to create agency column.
        agency_col(df_prefain)
        
        # Create a separate dataframe for the pre-validation error count.    
        # Convert to dataframe and set index=[0] to prevent scalar value error.
        preval_report = pd.DataFrame({'file_name': preval_file_name, 'preval_error_count': preval_count, 'preval_notification_date': date_preval}, index=[0])
        # Append all dataframes created by the for loop.
        df_preval = df_preval.append(preval_report)
        # Delete file from current directory.
        os.remove(preval_attachment)

    # Retrieve D2 error report messages. Set additional condition 'and len(re.findall...)==1' to filter out replies to forwarded messages.
    if 'DATA ACT D2 ERROR REPORT' in m.Subject.upper() and m.SenderName == 'OCFO - FMMI BI TEAM' or 'FW: DATA ACT D2 ERROR REPORT' in m.Subject.upper() and m.SenderName == 'Smith, John - OCFO, Washington, DC' and len(re.findall(r'From:', m.Body)) == 1:
        # Set filename and filepath and save to current directory.
        attachment = m.Attachments.Item(1)
        error_attachment = attachment.FileName
        attachment.SaveAsFile(os.getcwd() + '\\' + error_attachment)
        
        # Extract the reporting period date from the attachment filename and convert to date time format.
        date_error_text = error_attachment[-24:-5]
        # date_error_text = re.search(r'period (.*?)\.', m.Body).group(1) # Extract date from email body.
        date_error_replace = date_error_text.replace('-','')
        date_error = datetime.strptime(date_error_replace, '%Y%m%d%H%M%S')
        
        # Read into dataframe. 
        error_file = pd.read_excel(error_attachment, sheet_name='D2_Error_Records')
        # Remove whitespace from column names.
        error_file.rename(columns=lambda x: x.strip(), inplace=True)
        # Iterate through different versions of the source file column name to determine the correct one.
        for col in error_file.columns:
            if col=='Source File Name' or col=='FLEX_SOURCE_FILE_NAME':
                source_col = col
        
        # Get error count in the source file name column.
        error_file_count = error_file.loc[:,source_col].value_counts()
        # Extract the action date, FAIN and source file name columns.
        error_fain = error_file.loc[:,['Action Date', 'FAIN', 'Award Modification Amendment Number', 'URI', 'Federal Action Obligation', 'CFDA Number', source_col]]
        # Rename columns.
        error_fain.rename(columns={'Action Date': 'action_date', 'FAIN': 'fain', 'Award Modification Amendment Number': 'award_modification', 'URI': 'uri', 'Federal Action Obligation': 'obligation', 'CFDA Number': 'cfda_number', source_col: 'file_name'}, inplace=True)
        # Add an error date column.
        error_fain['D2_error_reporting_period'] = date_error
        # Append all dataframes.
        df_errfain = df_errfain.append(error_fain)
        # Apply agency_col function on the dataframe to create agency column.
        agency_col(df_errfain)
        
        # Create a separate dataframe for the D2 error count.
        # Convert index (i.e., filename) and value (i.e., count) to list.
        error_file_names = error_file_count.index.tolist()
        error_count = error_file_count.values.tolist()
        # List comprehension to assign reporting period date to each record.
        error_report_period = [date_error for f in error_file_names] 
        # Convert to dataframe.
        error_report = pd.DataFrame({'file_name': error_file_names, 'D2_error_count': error_count,'D2_error_reporting_period': error_report_period})
        # Append all dataframes created by the for loop.
        df_error = df_error.append(error_report)
        # Delete file from current directory.
        os.remove(error_attachment)
        
    # Retrieve D2 certification messages. Set additional condition 'and len(re.findall...)==1' to filter out replies to forwarded messages.
    if 'DATA ACT D2 CERTIFICATION REPORT' in m.Subject.upper() and m.SenderName == 'OCFO - FMMI BI TEAM' or 'FW: DATA ACT D2 CERTIFICATION REPORT' in m.Subject.upper() and m.SenderName == 'Smith, John - OCFO, Washington, DC' and len(re.findall(r'From:', m.Body)) == 1:       
        # Set filename and filepath and save to current directory. 
        attachment = m.Attachments.Item(1)
        cert_attachment = attachment.FileName
        attachment.SaveAsFile(os.getcwd() + '\\' + cert_attachment)
        
        # Extract the reporting period date from the attachment filename and convert to date time format.
        date_cert_text = cert_attachment[-24:-5]
        # date_cert_text = re.search(r'period (.*?)\.', m.Body).group(1) # Extract date from email body.
        date_cert_replace = date_cert_text.replace('-','')
        date_cert = datetime.strptime(date_cert_replace, '%Y%m%d%H%M%S')
        
        # Read into dataframe. 
        cert_file = pd.read_excel(cert_attachment, sheet_name='DATA ACT D2 CERTIFICATION REPOR')
        # Remove whitespace from column names.
        cert_file.rename(columns=lambda x: x.strip(), inplace=True)
        # Iterate through different versions of the source file column name to determine the correct one.
        for col in cert_file.columns:
            if col=='Source File Name' or col=='FLEX_SOURCE_FILE_NAME':
                source_col = col
        
        # Get cert count in the source file name column.
        cert_file_count = cert_file.loc[:,source_col].value_counts()
        # Extract the action date, FAIN and source file name columns.
        cert_fain = cert_file.loc[:,['ActionDate', 'FAIN', 'AwardModificationAmendmentNumber', 'URI', 'CFDA_Number', 'FederalActionObligation', source_col]]
        # Rename columns.
        cert_fain.rename(columns={'ActionDate': 'action_date', 'FAIN': 'fain', 'AwardModificationAmendmentNumber': 'award_modification', 'URI': 'uri', 'CFDA_Number': 'cfda_number', 'FederalActionObligation': 'obligation', source_col: 'file_name'}, inplace=True)
        # Add a cert date column.
        cert_fain['D2_cert_reporting_period'] = date_cert
        # Append all dataframes.
        df_certfain = df_certfain.append(cert_fain)
        # Apply agency_col function on the dataframe to create agency column.
        agency_col(df_certfain)
        
        # Create a separate dataframe for the D2 cert count.
        # Convert index (i.e., filename) and value (i.e., count) to list.
        cert_file_names = cert_file_count.index.tolist()
        cert_count = cert_file_count.values.tolist()
        # List comprehension to assign reporting period date to each record.
        cert_report_period = [date_cert for f in cert_file_names]
        # Convert to dataframe.
        cert_report = pd.DataFrame({'file_name': cert_file_names, 'D2_certification_count': cert_count,'D2_cert_reporting_period': cert_report_period})
        # Append all dataframes created by the for loop.
        df_cert = df_cert.append(cert_report)
        # Delete attachment from current directory.
        os.remove(cert_attachment)

### 4. Dataframe Processing
The final file will consist of four sheets created from the following dataframes: (1) a submission summary, (2) the prevalidation errors, and (3) D2 error records with a crosswalk to (4) the certified records.

#### a. Submission Summary

In [4]:
# Sort dataframes by descending date then drop duplicates (based on filename) and keep first instance (i.e., most recent based on sort).
# Create a sort and drop function.
def sort_drop(df,col1,col2):
    df.sort_values(by=[col1], ascending=False, inplace=True)
    df = df.drop_duplicates(subset=[col2], keep='first', inplace=True)
    
# Apply the sort_drop function on the dataframes.
sort_drop(df_submission, 'file_submission_notification_date', 'file_name')  
sort_drop(df_preval, 'preval_notification_date', 'file_name')
sort_drop(df_error, 'D2_error_reporting_period', 'file_name') 
sort_drop(df_cert, 'D2_cert_reporting_period', 'file_name')

# Merge dataframes on file_name.
df_comb1 = pd.merge(df_submission, df_preval, how='outer', on='file_name').fillna('')
df_comb2 = pd.merge(df_comb1, df_error, how='outer', on='file_name').fillna('')
df_summ_comb = pd.merge(df_comb2, df_cert, how='outer', on='file_name').fillna('')

# Apply the agency_col function on the dataframe to create an agency column.
agency_col(df_summ_comb)
# Rearrange columns.
df_summ_comb = df_summ_comb[['file_submission_notification_date', 'file_name', 'agency', 'preval_error_count', 'preval_notification_date', 'D2_error_count', 'D2_error_reporting_period', 'D2_certification_count', 'D2_cert_reporting_period']]

#### b. Prevalidation, Errors, and Certification Records

In [5]:
# Convert award_modification column to float to make format consistent for further procesing.
# Function to convert.
def con_num(df):
    df['award_modification'] = pd.to_numeric(df['award_modification'], errors='coerce')

# Apply con_num function.
con_num(df_prefain)
con_num(df_errfain)
con_num(df_certfain)

# Function to create a unique column for matching.
def key_col(df):
    df['ag_fain_mod_uri_cfda'] = df['agency_code'].astype(str) + df['fain'].astype(str) + df['award_modification'].astype(str) + df['uri'].astype(str) + df['cfda_number'].astype(str).str[:6]

# Apply the key_col function on the prevalidation, errors, and certification dataframes for subsequent cross-walking. 
key_col(df_prefain)
key_col(df_errfain)
key_col(df_certfain)

# Apply the sort_drop function on the dataframes to remove duplicates.
sort_drop(df_errfain, 'D2_error_reporting_period', 'ag_fain_mod_uri_cfda') 
sort_drop(df_certfain, 'D2_cert_reporting_period', 'ag_fain_mod_uri_cfda')

# Find matching records between preval-cert and error-cert dataframes.
df_pre_cert = pd.merge(df_prefain, df_certfain[['ag_fain_mod_uri_cfda', 'D2_cert_reporting_period', 'file_name']], how='left', on='ag_fain_mod_uri_cfda')
df_err_cert = pd.merge(df_errfain, df_certfain[['ag_fain_mod_uri_cfda', 'D2_cert_reporting_period', 'file_name']], how='left', on='ag_fain_mod_uri_cfda')

# Rename and rearrange columns.
df_pre_cert.rename(columns={'file_name_x': 'file_name_preval', 'file_name_y': 'file_name_cert'}, inplace=True)
df_err_cert.rename(columns={'file_name_x': 'file_name_error', 'file_name_y': 'file_name_cert'}, inplace=True)
df_pre_cert = df_pre_cert[['file_name_preval', 'notification_date', 'agency', 'fain', 'uri', 'action_date', 'award_modification', 'cfda_number', 'file_name_cert', 'D2_cert_reporting_period']]
df_err_cert = df_err_cert[['file_name_error', 'D2_error_reporting_period', 'agency', 'fain', 'uri', 'action_date', 'obligation', 'award_modification', 'cfda_number', 'file_name_cert', 'D2_cert_reporting_period']]
df_certfain = df_certfain[['file_name', 'D2_cert_reporting_period', 'agency', 'fain', 'uri', 'action_date', 'obligation', 'award_modification', 'cfda_number']]

### 5. Create xlsx File

In [6]:
# Create an xlsx file, one sheet for each dataframe.
# Set timestamp and file destination path.
timestr = time.strftime('%m_%d_%Y')
dest_path = os.getcwd() + '\\'

# Set file name and save to path.
file_name = 'D2_Files_Tracking_Report_'+ timestr + '.xlsx'
writer = pd.ExcelWriter(dest_path + file_name, engine='xlsxwriter')
df_summ_comb.to_excel(writer, 'Files_Summary', index=False)
df_pre_cert.to_excel(writer, 'Prevals', index=False)
df_err_cert.to_excel(writer, 'Errors', index=False)
df_certfain.to_excel(writer, 'Certs', index=False)
writer.save()

### 6. Notes

In [7]:
# To create a csv file, one file for each dataframe:
# file_name = 'dataframe_name'+ timestr + '.csv'
# df.to_csv(dest_path + file_name, index=False)

# To create a copy of the file in a different directory:
# shutil.copyfile(source path + file name, destination path + file name)

# Information can also be extracted from emails saved to a drive/folder using ExtractMsg (https://github.com/mattgwwalker/msg-extractor):
# import ExtractMsg
# msg = ExtractMsg.Message('message_subject.msg')
# print(msg.sender)
# print(msg.date)
# print(msg.body)